In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time

- #### Test Conditions

In [3]:
# Account info
username = 'username'
password = 'password'
AccountNumber = '123456789'        

# Start day is the first day of this month
# End day is today
from datetime import date
StartDate = date.today().strftime('%m/01/%Y')
EndDate = date.today().strftime('%m/%d/%Y')

- #### WebDriver

In [2]:
# Sends keys with delay in between
def send_keys_delay(element, word, delay):    
    for c in word:
        element.send_keys(c)
        time.sleep(delay)

In [4]:
# Firefox profile
profile = webdriver.FirefoxProfile()
profile.set_preference('browser.download.folderList', 2) # custom location
profile.set_preference('browser.download.manager.showWhenStarting', False)
profile.set_preference('browser.download.dir', '/tmp')
profile.set_preference('browser.helperApps.neverAsk.saveToDisk', 'text/csv')

In [5]:
# Firefox driver
driver = webdriver.Firefox(profile)

- #### AEPOhio.com Log in

In [6]:
# opens AEPOhio.com homepage
driver.get("https://www.aepohio.com")
assert "AEP Ohio" in driver.title

In [7]:
## Login
# clicks on login button and login form opens
elem = driver.find_element_by_xpath(
    "/html/body/form/span[2]/noindexfb/nav/div/div[2]/span/button").click()

# Username
# types the username in the username field
elem = driver.find_element_by_id ("ctl05_ctl04_ctl00_TbUserID")
elem.clear()
elem.send_keys(username)

#Password
# types the password in the password field
elem = driver.find_element_by_id ("ctl05_ctl04_ctl00_TbPassword")
elem.clear()
elem.send_keys(password)

# Login
# clicks on login button
elem = driver.find_element_by_id ("ctl05_ctl04_ctl00_BtnLogin").click()
# the page redirects to /account
# the usage data is in /account/usage

- #### Usage Data Extraction

In [8]:
# opens AEPOhio.com/Account/Usage
driver.get("https://www.aepohio.com/account/usage/")

In [10]:
# switch back to default content
driver.switch_to.default_content()

# select the account drop list and 
select = Select(driver.find_element_by_xpath(
    '//*[@id="cphContentMain_ctl00_ctl02_ctl00_DdlAccounts"]'))

In [11]:
## Dropdown List
# select a value in the drop list
select.select_by_value(AccountNumber)

In [12]:
## iFrame switch
# there is an iframe which shows the AJAX type of Google Analytics
# so first the driver swtichs to the dest iframe
# as it will show up by some delay, we used WebDriveWait
frame = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((
        By.XPATH,'//*[@id="energy_usage_trends-frame"]'))
)
driver.switch_to.frame(frame)

In [13]:
## Scroll Down
# scrolls down to the the three bar button
driver.execute_script("arguments[0].scrollIntoView(true);",
                      WebDriverWait(driver, 10).until(
                          EC.visibility_of_element_located((
                              By.XPATH,
                              '/html/body/div[3]/div[2]/div/div/div[2]/button'))))

In [14]:
## AJAX wait
# waits for the AJAX Google Analytics to load

# scrolls down to make it visible
driver.execute_script("arguments[0].scrollIntoView(true);",
                      WebDriverWait(driver, 10).until(
                          EC.visibility_of_element_located((
                              By.XPATH,
                              '/html/body/div[3]/div[2]/div/div/div[2]/button'))))

# waits for it to be clickable 
elem = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((
        By.XPATH,'/html/body/div[3]/div[2]/div/div/div[2]/button')))

# after a lot of tests, it seems still we need a short delay here(just half a second)
time.sleep(0.5)

# clicks on the 3 bars button at the upper right to open the list
elem.click()

In [15]:
# clicks on the 'Export All Data CSV'
# then it driver should wait for the dialog box 
elem = driver.find_element_by_xpath(
    '/html/body/div[3]/div[2]/div/div/div[2]/ul/li[5]/a').click()

In [16]:
# waits for the selection dialog box 'modal-Content'
elem = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((
        By.XPATH,'/html/body/div[5]/div/div'))
)

In [17]:
## Time
# From Date
elem = driver.find_element_by_xpath('//*[@id="green_button_form_start_green_button_date_range"]')
elem.clear()
# as it the field won't keep a date if you send it in one piece
# so I used a delayed send
send_keys_delay(elem, StartDate, 0.1)

# End Date
elem = driver.find_element_by_xpath('//*[@id="green_button_form_end_green_button_date_range"]')
elem.clear()
# as it the field won't keep a date if you send it in one piece
# so I used a delayed send
send_keys_delay(elem, EndDate, 0.1)

In [18]:
## Download
# clicks on download
elem = driver.find_element_by_xpath(
    '/html/body/div[5]/div/div/div[3]/button[2]').click()